In [201]:
import pandas as pd
pd.options.display.max_columns = 999
import numpy as np
#from numba import vectorize

from datetime import timedelta
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

%pylab inline


Populating the interactive namespace from numpy and matplotlib


In [202]:
import os
os.listdir('../')

['.DS_Store',
 '.git',
 '.ipynb_checkpoints',
 'data',
 'errors_and_model_selection',
 'features',
 'README.md',
 'short_term_forecasting',
 'visualization_notebooks']

In [203]:
def get_number_of_player(date):
    number_of_players = _df_pp[(_df_pp.Date == date)].player_id.count()
    return number_of_players

In [249]:
def only_predict(model, data, test_indexes, week):
    """
    Predict value by given:
        model
        data ~ train coumn/ test indexes
    """
    preds = model.predict(data.iloc[test_indexes, train_columns].values)
    df_new["week_{}_day_{}_lags_{}".format(week, target_column.split("_")[1], 
                                           len(train_columns))] = preds

def eval_score(model, data, test_indexes, week):
    """
    Evaluate score by 
    """
    preds = model.predict(data.iloc[test_indexes, train_columns].values)
    df_new["week_{}_day_{}_lags_{}".format(week, target_column.split("_")[1], 
                                           len(train_columns))] = preds
    return np.sqrt(mean_squared_error(data.loc[test_indexes, target_column].values, 
                                      preds))

def ts_cv(data, folds):
    """
    Time Series cross-validation
    
    """
    scores = []
    predictions = []
    for model, fold in enumerate(folds):
        lr = LinearRegression(n_jobs=-1)
        lr.fit(data.iloc[fold[0], train_columns], data.loc[fold[0], target_column])
        display(data.iloc[fold[0], train_columns])
        display(data.loc[fold[0], target_column])
        if model == 0:
            only_predict(lr, data, fold[1], model - 1)
        else:
            scores.append(eval_score(lr, data, fold[1], model - 1))
        
    return scores
        

In [250]:
df = pd.read_csv("../data/player_price.csv")
_df_ps = pd.read_csv("../data/player_stats.csv")

sample_subm = pd.read_csv("../data/kaggle_sample_submission.csv")

In [206]:
print("Read DataFrame")
df.head()

Read DataFrame


,timestamp,ps_price,xbox_price,player_id,player_name
0,1474588800000,0,0,1,Pelé
1,1474675200000,0,10000000,1,Pelé
2,1474761600000,0,0,1,Pelé
3,1474848000000,0,0,1,Pelé
4,1474934400000,0,10000000,1,Pelé


In [207]:
sample_subm["player_id"] = sample_subm["id"].apply(lambda x: int(x.split("_")[0]))
sample_subm["Date"] = sample_subm["id"].apply(lambda x: np.datetime64(x.split("_")[1]))

In [208]:
df = df[df["player_id"].isin(sample_subm["player_id"].unique())]
df['Date'] = pd.to_datetime(df['timestamp'], unit='ms')
_df_ps['Date'] = pd.to_datetime(_df_ps['added_on'])
df.drop(["timestamp", "ps_price", "player_name"], axis=1, inplace=True)

In [209]:
print("Get neccessery columns")
df.head()


Get neccessery columns


,xbox_price,player_id,Date
0,0,1,2016-09-23
1,10000000,1,2016-09-24
2,0,1,2016-09-25
3,0,1,2016-09-26
4,10000000,1,2016-09-27


In [210]:
#def create_added_player_feature():
#Add new feature number of player in game
num_players = pd.DataFrame()
_number_of_players = pd.DataFrame(_df_ps[['Date','player_id']].groupby('Date').player_id.count())
_number_of_players['num_players'] = _number_of_players['player_id']
_number_of_players.drop(["player_id"], axis=1, inplace=True)
_number_of_players = _number_of_players.reset_index()
display(_number_of_players.head())

,Date,num_players
0,2016-09-21,2774
1,2016-09-28,16
2,2016-09-30,24
3,2016-10-05,17
4,2016-10-07,2


In [211]:
df = df.merge(_number_of_players, how='left')
df = df.fillna(0)
display(df.head())

,xbox_price,player_id,Date,num_players
0,0,1,2016-09-23,0.0
1,10000000,1,2016-09-24,0.0
2,0,1,2016-09-25,0.0
3,0,1,2016-09-26,0.0
4,10000000,1,2016-09-27,0.0


In [212]:
dates_dict = {}
for i, date in enumerate(df["Date"].sort_values(ascending=False).unique()):
    dates_dict[date] = i
df["weekDate"] = df["Date"].map(dates_dict) // 7
df["wdayDate"] = df["Date"].map(dates_dict) % 7
df["wdayDate"] = df["wdayDate"].apply(lambda x: "day_" + str(x))
df["Date"] = df["Date"].astype(str)

In [213]:
print("Prepare new columns\n wdayDate - day of week\n weekDate - number of week")
df.head(10)

Prepare new columns
 wdayDate - day of week
 weekDate - number of week


,xbox_price,player_id,Date,num_players,weekDate,wdayDate
0,0,1,2016-09-23,0.0,38,day_6
1,10000000,1,2016-09-24,0.0,38,day_5
2,0,1,2016-09-25,0.0,38,day_4
3,0,1,2016-09-26,0.0,38,day_3
4,10000000,1,2016-09-27,0.0,38,day_2
5,0,1,2016-09-28,16.0,38,day_1
6,9500000,1,2016-09-29,0.0,38,day_0
7,8428000,1,2016-09-30,24.0,37,day_6
8,8459091,1,2016-10-01,0.0,37,day_5
9,7936136,1,2016-10-02,0.0,37,day_4


In [214]:
pp_df = pd.pivot_table(df, values="xbox_price", columns="wdayDate", index=["player_id", "weekDate"]).reset_index()
pp_df.columns = ['player_id', 'weekDate', 'day_0', 'day_1', 'day_2', 
              'day_3', 'day_4', 'day_5', 'day_6']

In [215]:
print("Pivot table for price")
display(pp_df.head())

Pivot table for price


,player_id,weekDate,day_0,day_1,day_2,day_3,day_4,day_5,day_6
0,1,0,3137143.0,3248943.0,3135387.0,3142655.0,3227893.0,3312852.0,3212762.0
1,1,1,3269308.0,3308667.0,3311636.0,3369791.0,3196091.0,3258917.0,3289895.0
2,1,2,3515515.0,3508952.0,3535840.0,3132091.0,3351619.0,3263857.0,3115720.0
3,1,3,3227500.0,3221877.0,3223901.0,3116277.0,3010988.0,3071354.0,2974671.0
4,1,4,2918560.0,2909955.0,2925575.0,2971810.0,2948080.0,2913674.0,2953371.0


In [70]:
np_df = pd.pivot_table(df, values="num_players", columns="wdayDate", index=["player_id", "weekDate"]).reset_index()
np_df.columns = ['player_id', 'weekDate', 'num_players_day_0', 'num_players_day_1', 'num_players_day_2', 
              'num_players_day_3', 'num_players_day_4', 'num_players_day_5', 'num_players_day_6']

In [71]:
print("Pivot table for number of players")
display(np_df.head())

Pivot table for number of players


,player_id,weekDate,num_players_day_0,num_players_day_1,num_players_day_2,num_players_day_3,num_players_day_4,num_players_day_5,num_players_day_6
0,1,0,0.0,0.0,4.0,0.0,0.0,0.0,1.0
1,1,1,22.0,12.0,41.0,0.0,1.0,0.0,0.0
2,1,2,22.0,7.0,25.0,0.0,0.0,0.0,23.0
3,1,3,0.0,18.0,14.0,0.0,0.0,3.0,21.0
4,1,4,1.0,17.0,27.0,0.0,0.0,4.0,0.0


In [216]:
temp = pp_df[pp_df["weekDate"] == 0]
temp[["weekDate", 'day_0', 'day_1', 'day_2', 
              'day_3', 'day_4', 'day_5', 'day_6']] = -1
pp_df = pd.concat([pp_df, temp])


In [73]:
_temp = np_df[np_df["weekDate"] == 0]
_temp[["weekDate", 'num_players_day_0', 'num_players_day_1', 'num_players_day_2', 
              'num_players_day_3', 'num_players_day_4', 'num_players_day_5', 'num_players_day_6']] = -1
np_df = pd.concat([np_df, _temp])

In [217]:
pp_df.head()

,player_id,weekDate,day_0,day_1,day_2,day_3,day_4,day_5,day_6
0,1,0,3137143.0,3248943.0,3135387.0,3142655.0,3227893.0,3312852.0,3212762.0
1,1,1,3269308.0,3308667.0,3311636.0,3369791.0,3196091.0,3258917.0,3289895.0
2,1,2,3515515.0,3508952.0,3535840.0,3132091.0,3351619.0,3263857.0,3115720.0
3,1,3,3227500.0,3221877.0,3223901.0,3116277.0,3010988.0,3071354.0,2974671.0
4,1,4,2918560.0,2909955.0,2925575.0,2971810.0,2948080.0,2913674.0,2953371.0


In [75]:
np_df.head()

,player_id,weekDate,num_players_day_0,num_players_day_1,num_players_day_2,num_players_day_3,num_players_day_4,num_players_day_5,num_players_day_6
0,1,0,0.0,0.0,4.0,0.0,0.0,0.0,1.0
1,1,1,22.0,12.0,41.0,0.0,1.0,0.0,0.0
2,1,2,22.0,7.0,25.0,0.0,0.0,0.0,23.0
3,1,3,0.0,18.0,14.0,0.0,0.0,3.0,21.0
4,1,4,1.0,17.0,27.0,0.0,0.0,4.0,0.0


In [76]:
temp.head()

,player_id,weekDate,day_0,day_1,day_2,day_3,day_4,day_5,day_6
0,1,-1,-1,-1,-1,-1,-1,-1,-1
39,2,-1,-1,-1,-1,-1,-1,-1,-1
78,3,-1,-1,-1,-1,-1,-1,-1,-1
117,4,-1,-1,-1,-1,-1,-1,-1,-1
156,5,-1,-1,-1,-1,-1,-1,-1,-1


In [218]:
def create_lag_features(pp_df, lag_week, feature='day_'):
    temp = pp_df.copy()
    temp['weekDate'] = pp_df['weekDate'].values - lag_week
    cols = []
    for i in range(7):
        cols.append("lag_{}_{}".format(lag_week, feature + str(i)))
        temp.rename(columns={"{}".format(feature + str(i)): cols[-1]}, inplace=True)
    return pp_df.merge(temp[["weekDate", "player_id"] + cols], 
                                   how='left', on=["weekDate", "player_id"])

In [241]:
for lag in range(1, 5):
    pp_df = create_lag_features(pp_df,lag)

In [242]:
pp_df.tail()

,player_id,weekDate,day_0,day_1,day_2,day_3,day_4,day_5,day_6,lag_1_day_0_x,lag_1_day_1_x,lag_1_day_2_x,lag_1_day_3_x,lag_1_day_4_x,lag_1_day_5_x,lag_1_day_6_x,lag_2_day_0_x,lag_2_day_1_x,lag_2_day_2_x,lag_2_day_3_x,lag_2_day_4_x,lag_2_day_5_x,lag_2_day_6_x,lag_3_day_0_x,lag_3_day_1_x,lag_3_day_2_x,lag_3_day_3_x,lag_3_day_4_x,lag_3_day_5_x,lag_3_day_6_x,lag_4_day_0_x,lag_4_day_1_x,lag_4_day_2_x,lag_4_day_3_x,lag_4_day_4_x,lag_4_day_5_x,lag_4_day_6_x,lag_1_day_0_y,lag_1_day_0_y,lag_1_day_1_y,lag_1_day_1_y,lag_1_day_2_y,lag_1_day_2_y,lag_1_day_3_y,lag_1_day_3_y,lag_1_day_4_y,lag_1_day_4_y,lag_1_day_5_y,lag_1_day_5_y,lag_1_day_6_y,lag_1_day_6_y,lag_2_day_0_y,lag_2_day_0_y,lag_2_day_1_y,lag_2_day_1_y,lag_2_day_2_y,lag_2_day_2_y,lag_2_day_3_y,lag_2_day_3_y,lag_2_day_4_y,lag_2_day_4_y,lag_2_day_5_y,lag_2_day_5_y,lag_2_day_6_y,lag_2_day_6_y,lag_3_day_0_y,lag_3_day_0_y,lag_3_day_1_y,lag_3_day_1_y,lag_3_day_2_y,lag_3_day_2_y,lag_3_day_3_y,lag_3_day_3_y,lag_3_day_4_y,lag_3_day_4_y,lag_3_day_5_y,lag_3_day_5_y,lag_3_day_6_y,lag_3_day_6_y,lag_4_day_0_y,lag_4_day_0_y,lag_4_day_1_y,lag_4_day_1_y,lag_4_day_2_y,lag_4_day_2_y,lag_4_day_3_y,lag_4_day_3_y,lag_4_day_4_y,lag_4_day_4_y,lag_4_day_5_y,lag_4_day_5_y,lag_4_day_6_y,lag_4_day_6_y
36845,18341,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,13390.0,13057.0,12575.0,12444.0,13389.0,13444.0,13100.0,12563.0,12250.0,12750.0,11417.0,12500.0,12917.0,13350.0,14500.0,13167.0,12607.0,12875.0,12042.0,12156.0,12821.0,12875.0,12673.0,11793.0,11655.0,12345.0,11948.0,11952.0,13390.0,12563.0,13057.0,12250.0,12575.0,12750.0,12444.0,11417.0,13389.0,12500.0,13444.0,12917.0,13100.0,13350.0,12563.0,12875.0,12250.0,12673.0,12750.0,11793.0,11417.0,11655.0,12500.0,12345.0,12917.0,11948.0,13350.0,11952.0,14500.0,12761.0,13167.0,13230.0,12607.0,14409.0,12875.0,13640.0,12042.0,14480.0,12156.0,13689.0,12821.0,12022.0,12875.0,13557.0,12673.0,13672.0,11793.0,14406.0,11655.0,13302.0,12345.0,13742.0,11948.0,13705.0,11952.0,11767.0
36846,18342,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,12229.0,12663.0,12273.0,12444.0,12389.0,12000.0,11958.0,11833.0,12333.0,12750.0,12167.0,12417.0,11667.0,12600.0,12025.0,11950.0,11906.0,11833.0,11100.0,11750.0,11571.0,11653.0,11942.0,11848.0,11957.0,12190.0,12500.0,12038.0,12229.0,11833.0,12663.0,12333.0,12273.0,12750.0,12444.0,12167.0,12389.0,12417.0,12000.0,11667.0,11958.0,12600.0,11833.0,11653.0,12333.0,11942.0,12750.0,11848.0,12167.0,11957.0,12417.0,12190.0,11667.0,12500.0,12600.0,12038.0,12025.0,12174.0,11950.0,12431.0,11906.0,12647.0,11833.0,12853.0,11100.0,13396.0,11750.0,12705.0,11571.0,14168.0,11653.0,14934.0,11942.0,15221.0,11848.0,15061.0,11957.0,15057.0,12190.0,14831.0,12500.0,14809.0,12038.0,13160.0
36847,18344,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,12364.0,12250.0,11900.0,12111.0,12694.0,11875.0,12042.0,12083.0,11417.0,11000.0,11583.0,12333.0,12833.0,10650.0,11375.0,11250.0,10969.0,11875.0,12708.0,12786.0,12143.0,13224.0,13250.0,11871.0,14129.0,12974.0,13690.0,13837.0,12364.0,12083.0,12250.0,11417.0,11900.0,11000.0,12111.0,11583.0,12694.0,12333.0,11875.0,12833.0,12042.0,10650.0,12083.0,13224.0,11417.0,13250.0,11000.0,11871.0,11583.0,14129.0,12333.0,12974.0,12833.0,13690.0,10650.0,13837.0,11375.0,11533.0,11250.0,10871.0,10969.0,11060.0,11875.0,12089.0,12708.0,12129.0,12786.0,11633.0,12143.0,12213.0,13224.0,13296.0,13250.0,11457.0,11871.0,11342.0,14129.0,11486.0,12974.0,11696.0,13690.0,11023.0,13837.0,11229.0
36848,18345,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,56515.0,56932.0,56545.0,56722.0,58833.0,62444.0,64100.0,68600.0,72333.0,65500.0,55000.0,55167.0,55333.0,63100.0,60450.0,60583.0,61611.0,59286.0,59313.0,61500.0,57100.0,55472.0,50250.0,50810.0,49405.0,54457.0,52940.0,51038.0,56515.0,68600.0,56932.0,72333.0,56545.0,65500.0,56722.0,55000.0,58833.0,55167.0,62444.0,55333.0,64100.0,63100.0,68600.0,55472.0,72333.0,50250.0,65500.0,50810.0,55000.0,49405.0,55167.0,54457.0,55333.0,52940.0,63100.0,51038.0,60450.0,56431.0,60583.0,52605.0,61611.0,58372.0,59286.0,40487.0,59313.0,38846.0,61500.0,37910.0,57

In [81]:
for lag in range(1, 5):
    np_df = create_lag_features(np_df, lag, feature = 'num_players_day_')

In [82]:
np_df.drop(['player_id','weekDate'], axis=1, inplace=True)
#pp_df.columns 

In [83]:
display(np_df.head())
display(pp_df.head())

,num_players_day_0,num_players_day_1,num_players_day_2,num_players_day_3,num_players_day_4,num_players_day_5,num_players_day_6,lag_1_num_players_day_0,lag_1_num_players_day_1,lag_1_num_players_day_2,lag_1_num_players_day_3,lag_1_num_players_day_4,lag_1_num_players_day_5,lag_1_num_players_day_6,lag_2_num_players_day_0,lag_2_num_players_day_1,lag_2_num_players_day_2,lag_2_num_players_day_3,lag_2_num_players_day_4,lag_2_num_players_day_5,lag_2_num_players_day_6,lag_3_num_players_day_0,lag_3_num_players_day_1,lag_3_num_players_day_2,lag_3_num_players_day_3,lag_3_num_players_day_4,lag_3_num_players_day_5,lag_3_num_players_day_6,lag_4_num_players_day_0,lag_4_num_players_day_1,lag_4_num_players_day_2,lag_4_num_players_day_3,lag_4_num_players_day_4,lag_4_num_players_day_5,lag_4_num_players_day_6
0,0.0,0.0,4.0,0.0,0.0,0.0,1.0,22.0,12.0,41.0,0.0,1.0,0.0,0.0,22.0,7.0,25.0,0.0,0.0,0.0,23.0,0.0,18.0,14.0,0.0,0.0,3.0,21.0,1.0,17.0,27.0,0.0,0.0,4.0,0.0
1,22.0,12.0,41.0,0.0,1.0,0.0,0.0,22.0,7.0,25.0,0.0,0.0,0.0,23.0,0.0,18.0,14.0,0.0,0.0,3.0,21.0,1.0,17.0,27.0,0.0,0.0,4.0,0.0,0.0,38.0,2.0,2.0,0.0,3.0,21.0
2,22.0,7.0,25.0,0.0,0.0,0.0,23.0,0.0,18.0,14.0,0.0,0.0,3.0,21.0,1.0,17.0,27.0,0.0,0.0,4.0,0.0,0.0,38.0,2.0,2.0,0.0,3.0,21.0,0.0,39.0,0.0,0.0,0.0,5.0,0.0
3,0.0,18.0,14.0,0.0,0.0,3.0,21.0,1.0,17.0,27.0,0.0,0.0,4.0,0.0,0.0,38.0,2.0,2.0,0.0,3.0,21.0,0.0,39.0,0.0,0.0,0.0,5.0,0.0,1.0,18.0,0.0,0.0,0.0,8.0,0.0
4,1.0,17.0,27.0,0.0,0.0,4.0,0.0,0.0,38.0,2.0,2.0,0.0,3.0,21.0,0.0,39.0,0.0,0.0,0.0,5.0,0.0,1.0,18.0,0.0,0.0,0.0,8.0,0.0,0.0,15.0,0.0,2.0,0.0,2.0,21.0


,player_id,weekDate,day_0,day_1,day_2,day_3,day_4,day_5,day_6,lag_1_day_0,lag_1_day_1,lag_1_day_2,lag_1_day_3,lag_1_day_4,lag_1_day_5,lag_1_day_6,lag_2_day_0,lag_2_day_1,lag_2_day_2,lag_2_day_3,lag_2_day_4,lag_2_day_5,lag_2_day_6,lag_3_day_0,lag_3_day_1,lag_3_day_2,lag_3_day_3,lag_3_day_4,lag_3_day_5,lag_3_day_6,lag_4_day_0,lag_4_day_1,lag_4_day_2,lag_4_day_3,lag_4_day_4,lag_4_day_5,lag_4_day_6
0,1,0,3137143.0,3248943.0,3135387.0,3142655.0,3227893.0,3312852.0,3212762.0,3269308.0,3308667.0,3311636.0,3369791.0,3196091.0,3258917.0,3289895.0,3515515.0,3508952.0,3535840.0,3132091.0,3351619.0,3263857.0,3115720.0,3227500.0,3221877.0,3223901.0,3116277.0,3010988.0,3071354.0,2974671.0,2918560.0,2909955.0,2925575.0,2971810.0,2948080.0,2913674.0,2953371.0
1,1,1,3269308.0,3308667.0,3311636.0,3369791.0,3196091.0,3258917.0,3289895.0,3515515.0,3508952.0,3535840.0,3132091.0,3351619.0,3263857.0,3115720.0,3227500.0,3221877.0,3223901.0,3116277.0,3010988.0,3071354.0,2974671.0,2918560.0,2909955.0,2925575.0,2971810.0,2948080.0,2913674.0,2953371.0,2943406.0,3035780.0,3048385.0,3026164.0,3191148.0,3125109.0,3150129.0
2,1,2,3515515.0,3508952.0,3535840.0,3132091.0,3351619.0,3263857.0,3115720.0,3227500.0,3221877.0,3223901.0,3116277.0,3010988.0,3071354.0,2974671.0,2918560.0,2909955.0,2925575.0,2971810.0,2948080.0,2913674.0,2953371.0,2943406.0,3035780.0,3048385.0,3026164.0,3191148.0,3125109.0,3150129.0,3149528.0,3249785.0,3374493.0,3401396.0,3461229.0,3484594.0,3397083.0
3,1,3,3227500.0,3221877.0,3223901.0,3116277.0,3010988.0,3071354.0,2974671.0,2918560.0,2909955.0,2925575.0,2971810.0,2948080.0,2913674.0,2953371.0,2943406.0,3035780.0,3048385.0,3026164.0,3191148.0,3125109.0,3150129.0,3149528.0,3249785.0,3374493.0,3401396.0,3461229.0,3484594.0,3397083.0,3444301.0,3409347.0,3552750.0,3381701.0,3389443.0,3345167.0,3523771.0
4,1,4,2918560.0,2909955.0,2925575.0,2971810.0,2948080.0,2913674.0,2953371.0,2943406.0,3035780.0,3048385.0,3026164.0,3191148.0,3125109.0,3150129.0,3149528.0,3249785.0,3374493.0,3401396.0,3461229.0,3484594.0,3397083.0,3444301.0,3409347.0,3552750.0,3381701.0,3389443.0,3345167.0,3523771.0,3695750.0,3762972.0,3823042.0,3849000.0,4016643.0,4166220.0,4117158.0


In [84]:
__df = pd.concat([pp_df, np_df], axis=1, copy=False)

In [85]:
__df.head()

,player_id,weekDate,day_0,day_1,day_2,day_3,day_4,day_5,day_6,lag_1_day_0,lag_1_day_1,lag_1_day_2,lag_1_day_3,lag_1_day_4,lag_1_day_5,lag_1_day_6,lag_2_day_0,lag_2_day_1,lag_2_day_2,lag_2_day_3,lag_2_day_4,lag_2_day_5,lag_2_day_6,lag_3_day_0,lag_3_day_1,lag_3_day_2,lag_3_day_3,lag_3_day_4,lag_3_day_5,lag_3_day_6,lag_4_day_0,lag_4_day_1,lag_4_day_2,lag_4_day_3,lag_4_day_4,lag_4_day_5,lag_4_day_6,num_players_day_0,num_players_day_1,num_players_day_2,num_players_day_3,num_players_day_4,num_players_day_5,num_players_day_6,lag_1_num_players_day_0,lag_1_num_players_day_1,lag_1_num_players_day_2,lag_1_num_players_day_3,lag_1_num_players_day_4,lag_1_num_players_day_5,lag_1_num_players_day_6,lag_2_num_players_day_0,lag_2_num_players_day_1,lag_2_num_players_day_2,lag_2_num_players_day_3,lag_2_num_players_day_4,lag_2_num_players_day_5,lag_2_num_players_day_6,lag_3_num_players_day_0,lag_3_num_players_day_1,lag_3_num_players_day_2,lag_3_num_players_day_3,lag_3_num_players_day_4,lag_3_num_players_day_5,lag_3_num_players_day_6,lag_4_num_players_day_0,lag_4_num_players_day_1,lag_4_num_players_day_2,lag_4_num_players_day_3,lag_4_num_players_day_4,lag_4_num_players_day_5,lag_4_num_players_day_6
0,1,0,3137143.0,3248943.0,3135387.0,3142655.0,3227893.0,3312852.0,3212762.0,3269308.0,3308667.0,3311636.0,3369791.0,3196091.0,3258917.0,3289895.0,3515515.0,3508952.0,3535840.0,3132091.0,3351619.0,3263857.0,3115720.0,3227500.0,3221877.0,3223901.0,3116277.0,3010988.0,3071354.0,2974671.0,2918560.0,2909955.0,2925575.0,2971810.0,2948080.0,2913674.0,2953371.0,0.0,0.0,4.0,0.0,0.0,0.0,1.0,22.0,12.0,41.0,0.0,1.0,0.0,0.0,22.0,7.0,25.0,0.0,0.0,0.0,23.0,0.0,18.0,14.0,0.0,0.0,3.0,21.0,1.0,17.0,27.0,0.0,0.0,4.0,0.0
1,1,1,3269308.0,3308667.0,3311636.0,3369791.0,3196091.0,3258917.0,3289895.0,3515515.0,3508952.0,3535840.0,3132091.0,3351619.0,3263857.0,3115720.0,3227500.0,3221877.0,3223901.0,3116277.0,3010988.0,3071354.0,2974671.0,2918560.0,2909955.0,2925575.0,2971810.0,2948080.0,2913674.0,2953371.0,2943406.0,3035780.0,3048385.0,3026164.0,3191148.0,3125109.0,3150129.0,22.0,12.0,41.0,0.0,1.0,0.0,0.0,22.0,7.0,25.0,0.0,0.0,0.0,23.0,0.0,18.0,14.0,0.0,0.0,3.0,21.0,1.0,17.0,27.0,0.0,0.0,4.0,0.0,0.0,38.0,2.0,2.0,0.0,3.0,21.0
2,1,2,3515515.0,3508952.0,3535840.0,3132091.0,3351619.0,3263857.0,3115720.0,3227500.0,3221877.0,3223901.0,3116277.0,3010988.0,3071354.0,2974671.0,2918560.0,2909955.0,2925575.0,2971810.0,2948080.0,2913674.0,2953371.0,2943406.0,3035780.0,3048385.0,3026164.0,3191148.0,3125109.0,3150129.0,3149528.0,3249785.0,3374493.0,3401396.0,3461229.0,3484594.0,3397083.0,22.0,7.0,25.0,0.0,0.0,0.0,23.0,0.0,18.0,14.0,0.0,0.0,3.0,21.0,1.0,17.0,27.0,0.0,0.0,4.0,0.0,0.0,38.0,2.0,2.0,0.0,3.0,21.0,0.0,39.0,0.0,0.0,0.0,5.0,0.0
3,1,3,3227500.0,3221877.0,3223901.0,3116277.0,3010988.0,3071354.0,2974671.0,2918560.0,2909955.0,2925575.0,2971810.0,2948080.0,2913674.0,2953371.0,2943406.0,3035780.0,3048385.0,3026164.0,3191148.0,3125109.0,3150129.0,3149528.0,3249785.0,3374493.0,3401396.0,3461229.0,3484594.0,3397083.0,3444301.0,3409347.0,3552750.0,3381701.0,3389443.0,3345167.0,3523771.0,0.0,18.0,14.0,0.0,0.0,3.0,21.0,1.0,17.0,27.0,0.0,0.0,4.0,0.0,0.0,38.0,2.0,2.0,0.0,3.0,21.0,0.0,39.0,0.0,0.0,0.0,5.0,0.0,1.0,18.0,0.0,0.0,0.0,8.0,0.0
4,1,4,2918560.0,2909955.0,2925575.0,2971810.0,2948080.0,2913674.0,2953371.0,2943406.0,3035780.0,3048385.0,3026164.0,3191148.0,3125109.0,3150129.0,3149528.0,3249785.0,3374493.0,3401396.0,3461229.0,3484594.0,3397083.0,3444301.0,3409347.0,3552750.0,3381701.0,3389443.0,3345167.0,3523771.0,3695750.0,3762972.0,3823042.0,3849000.0,4016643.0,4166220.0,4117158.0,1.0,17.0,27.0,0.0,0.0,4.0,0.0,0.0,38.0,2.0,2.0,0.0,3.0,21.0,0.0,39.0,0.0,0.0,0.0,5.0,0.0,1.0,18.0,0.0,0.0,0.0,8.0,0.0,0.0,15.0,0.0,2.0,0.0,2.0,21.0


In [99]:
[i for i in enumerate(__df.columns.tolist())]

[(0, 'player_id'),
 (1, 'weekDate'),
 (2, 'day_0'),
 (3, 'day_1'),
 (4, 'day_2'),
 (5, 'day_3'),
 (6, 'day_4'),
 (7, 'day_5'),
 (8, 'day_6'),
 (9, 'lag_1_day_0'),
 (10, 'lag_1_day_1'),
 (11, 'lag_1_day_2'),
 (12, 'lag_1_day_3'),
 (13, 'lag_1_day_4'),
 (14, 'lag_1_day_5'),
 (15, 'lag_1_day_6'),
 (16, 'lag_2_day_0'),
 (17, 'lag_2_day_1'),
 (18, 'lag_2_day_2'),
 (19, 'lag_2_day_3'),
 (20, 'lag_2_day_4'),
 (21, 'lag_2_day_5'),
 (22, 'lag_2_day_6'),
 (23, 'lag_3_day_0'),
 (24, 'lag_3_day_1'),
 (25, 'lag_3_day_2'),
 (26, 'lag_3_day_3'),
 (27, 'lag_3_day_4'),
 (28, 'lag_3_day_5'),
 (29, 'lag_3_day_6'),
 (30, 'lag_4_day_0'),
 (31, 'lag_4_day_1'),
 (32, 'lag_4_day_2'),
 (33, 'lag_4_day_3'),
 (34, 'lag_4_day_4'),
 (35, 'lag_4_day_5'),
 (36, 'lag_4_day_6'),
 (37, 'num_players_day_0'),
 (38, 'num_players_day_1'),
 (39, 'num_players_day_2'),
 (40, 'num_players_day_3'),
 (41, 'num_players_day_4'),
 (42, 'num_players_day_5'),
 (43, 'num_players_day_6'),
 (44, 'lag_1_num_players_day_0'),
 (45, 'lag_1_

In [243]:
validation_weeks = [-1, 0]

In [244]:
def create_validation(df,week):
    """
    Return indexes for train and test data set
    -1 - for predict data set
    0 - for train/test dataset
    """
    return df[(df["weekDate"] > week) & (df["weekDate"] < week + 2)].index, df[df["weekDate"] == week].index

In [245]:
validation = []
for week in validation_weeks:
    validation.append(create_validation(pp_df, week))


In [246]:
print("Create Validation Week:\n consist from two parts, train and test data frame")
validation

Create Validation Week:
 consist from two parts, train and test data frame


[(Int64Index([    0,    39,    78,   117,   156,   195,   234,   273,   312,
                351,
              ...
              35507, 35516, 35525, 35534, 35543, 35552, 35561, 35570, 35579,
              35588],
             dtype='int64', length=1253),
  Int64Index([35597, 35598, 35599, 35600, 35601, 35602, 35603, 35604, 35605,
              35606,
              ...
              36840, 36841, 36842, 36843, 36844, 36845, 36846, 36847, 36848,
              36849],
             dtype='int64', length=1253)),
 (Int64Index([    1,    40,    79,   118,   157,   196,   235,   274,   313,
                352,
              ...
              35508, 35517, 35526, 35535, 35544, 35553, 35562, 35571, 35580,
              35589],
             dtype='int64', length=1253),
  Int64Index([    0,    39,    78,   117,   156,   195,   234,   273,   312,
                351,
              ...
              35507, 35516, 35525, 35534, 35543, 35552, 35561, 35570, 35579,
              35588],
             

In [252]:
scores_cv = []
best_days = [1,1,1,1,1,1,2]
lags_values = range(1,7)
df_new = pd.DataFrame([])
for day in range(7):
    scores_cv.append([])
    for n_lag in lags_values:
        train_columns = list(range(9,9+n_lag))+
        print(train_columns)
        target_column = "day_{}".format(day)
        scores_cv[-1].append(ts_cv(pp_df.fillna(0), validation))

[9]


,lag_1_day_0_x
0,3269308.0
39,779230.0
78,406806.0
117,554714.0
156,374929.0
195,205538.0
234,69375.0
273,157449.0
312,195381.0
351,670769.0


0        3137143.0
39        742514.0
78        378507.0
117       551679.0
156       351241.0
195       233931.0
234        63944.0
273       139070.0
312       188076.0
351       654586.0
390       223250.0
429       639107.0
468       139414.0
507       377250.0
546       204357.0
585       176036.0
624       170483.0
663       184414.0
702        95401.0
741       122396.0
780        57596.0
819        36894.0
858       218897.0
897       236069.0
936        47569.0
975      1564821.0
1014      679286.0
1053      125536.0
1092      155448.0
1131      185607.0
           ...    
35319      30272.0
35329      31864.0
35339      17721.0
35349      17854.0
35359      15404.0
35369      22169.0
35379      47652.0
35389      20313.0
35399    2148424.0
35408     489594.0
35417     163706.0
35426      69485.0
35435     279606.0
35444     372147.0
35453     213765.0
35462     475494.0
35471      18700.0
35480     100838.0
35489      79602.0
35498      25735.0
35507      21750.0
35516      5

,lag_1_day_0_x
1,3515515.0
40,789868.0
79,396375.0
118,581176.0
157,396515.0
196,221939.0
235,76944.0
274,160383.0
313,197424.0
352,671727.0


1        3269308.0
40        779230.0
79        406806.0
118       554714.0
157       374929.0
196       205538.0
235        69375.0
274       157449.0
313       195381.0
352       670769.0
391       238308.0
430       663500.0
469       143308.0
508       389462.0
547       213857.0
586       205857.0
625       180077.0
664       184538.0
703       107099.0
742       131482.0
781        60212.0
820        40107.0
859       195308.0
898       247462.0
937        55604.0
976      1535923.0
1015      718286.0
1054      130462.0
1093      181231.0
1132      200429.0
           ...    
35320      33417.0
35330      33750.0
35340      14167.0
35350      15750.0
35360      27167.0
35370      22000.0
35380      53375.0
35390      24941.0
35400    2392250.0
35409     501000.0
35418     115750.0
35427      80250.0
35436     284667.0
35445     415500.0
35454     255500.0
35463     451953.0
35472      20625.0
35481     101333.0
35490      94286.0
35499      27188.0
35508      24583.0
35517      6

...

[9, 10, 11, 12, 13, 14]


,lag_1_day_0_x,lag_1_day_1_x,lag_1_day_2_x,lag_1_day_3_x,lag_1_day_4_x,lag_1_day_5_x
0,3269308.0,3308667.0,3311636.0,3369791.0,3196091.0,3258917.0
39,779230.0,787050.0,789865.0,786191.0,795044.0,796655.0
78,406806.0,402856.0,411600.0,401496.0,409529.0,417459.0
117,554714.0,556462.0,550667.0,553500.0,553917.0,590167.0
156,374929.0,386846.0,387667.0,379167.0,377909.0,385083.0
195,205538.0,211308.0,217667.0,215000.0,231167.0,221750.0
234,69375.0,68000.0,0.0,67250.0,71250.0,68250.0
273,157449.0,157682.0,164667.0,165133.0,168517.0,166890.0
312,195381.0,195590.0,195156.0,195301.0,198045.0,200281.0
351,670769.0,678923.0,663000.0,663333.0,672333.0,676917.0


0        3212762.0
39        784590.0
78        404118.0
117       555952.0
156       383381.0
195       219238.0
234        68667.0
273       156824.0
312       193273.0
351       663333.0
390       230524.0
429       656571.0
468       142952.0
507       394952.0
546       213143.0
585       194286.0
624       182304.0
663       186762.0
702       105733.0
741       132296.0
780        61975.0
819        39368.0
858       194905.0
897       245714.0
936        55944.0
975      1581333.0
1014      729810.0
1053      130619.0
1092      171571.0
1131      201619.0
           ...    
35319      35458.0
35329      35500.0
35339      15417.0
35349      15650.0
35359      15050.0
35369      22917.0
35379      53400.0
35389      24167.0
35399    2306200.0
35408     509167.0
35417     131067.0
35426      77500.0
35435     277833.0
35444     410800.0
35453     242000.0
35462     448778.0
35471      19150.0
35480     107167.0
35489      98744.0
35498      26250.0
35507      23300.0
35516      6

,lag_1_day_0_x,lag_1_day_1_x,lag_1_day_2_x,lag_1_day_3_x,lag_1_day_4_x,lag_1_day_5_x
1,3515515.0,3508952.0,3535840.0,3132091.0,3351619.0,3263857.0
40,789868.0,787597.0,785222.0,772681.0,770208.0,742903.0
79,396375.0,392701.0,392306.0,395174.0,395063.0,392674.0
118,581176.0,574900.0,590385.0,612136.0,554571.0,551821.0
157,396515.0,391000.0,379038.0,370182.0,368095.0,393214.0
196,221939.0,220550.0,224962.0,233905.0,234000.0,240741.0
235,76944.0,73750.0,78214.0,76500.0,75200.0,81063.0
274,160383.0,164733.0,164149.0,164863.0,165033.0,168242.0
313,197424.0,195224.0,194331.0,191357.0,189403.0,186688.0
352,671727.0,669143.0,678923.0,677318.0,675381.0,687357.0


1        3289895.0
40        796667.0
79        409258.0
118       602222.0
157       394579.0
196       225947.0
235        75500.0
274       167738.0
313       200163.0
352       668000.0
391       241889.0
430       651778.0
469       144444.0
508       394222.0
547       227211.0
586       194105.0
625       196556.0
664       192684.0
703       124967.0
742       136071.0
781        66400.0
820        42583.0
859       214053.0
898       260333.0
937        57234.0
976      1598389.0
1015      727105.0
1054      138833.0
1093      183316.0
1132      211111.0
           ...    
35320      48375.0
35330      39850.0
35340      14300.0
35350      13300.0
35360      17500.0
35370      19050.0
35380      66200.0
35390      21550.0
35400    2154000.0
35409     562600.0
35418     134400.0
35427      83600.0
35436     306625.0
35445     445200.0
35454     277429.0
35463     453722.0
35472      22900.0
35481     103400.0
35490      92992.0
35499      29600.0
35508      24250.0
35517      7

In [253]:
print("Perform Validation and count best days")
scores_cv

Perform Validation and count best days


[[[23455.714448867482],
  [24209.597237217316],
  [27590.305916840654],
  [39472.638261874621],
  [35786.23883923189],
  [37013.318774704232]],
 [[26219.322585239552],
  [29580.886816417584],
  [37810.118151687195],
  [51565.520088266348],
  [50291.557040672982],
  [52432.243404159133]],
 [[18547.790922537846],
  [49399.350480330046],
  [52190.847088149087],
  [57869.150033191705],
  [50537.078583904651],
  [48919.779772073416]],
 [[24274.124437572107],
  [25354.711090965073],
  [25853.164281099715],
  [26149.418858381581],
  [26970.011935267554],
  [29621.729842702403]],
 [[20247.930087529872],
  [20940.945889392828],
  [21127.448919004019],
  [37514.930170232903],
  [35748.978248618921],
  [35821.953553315485]],
 [[14737.06470712816],
  [14937.722503385712],
  [41786.805525165735],
  [19011.280452128121],
  [22808.587241201087],
  [23896.844222998101]],
 [[10722.567138349788],
  [10399.486628815799],
  [50273.579959260707],
  [15420.278896925496],
  [15180.849385435906],
  [15647.596

In [232]:
lags_values

range(1, 7)

In [233]:
best_days = [lags_values[i] for i in np.argmin(np.mean(np.array(scores_cv), axis=2), axis=1)]

In [234]:
best_days

[1, 1, 1, 6, 6, 1, 1]

In [235]:
srore = np.mean(np.array([scores_cv[i][best_days[i]-1] for i in range(0,len(scores_cv))]))

In [236]:
srore

9.4421103606227571e-11

In [237]:
np.mean([np.mean(np.array(scores_cv), axis=2)[i,j-1] for i, j in enumerate(best_days)])

9.4421103606227571e-11

In [238]:
pp_df

,player_id,weekDate,day_0,day_1,day_2,day_3,day_4,day_5,day_6,lag_1_day_0,lag_1_day_1,lag_1_day_2,lag_1_day_3,lag_1_day_4,lag_1_day_5,lag_1_day_6,lag_2_day_0,lag_2_day_1,lag_2_day_2,lag_2_day_3,lag_2_day_4,lag_2_day_5,lag_2_day_6,lag_3_day_0,lag_3_day_1,lag_3_day_2,lag_3_day_3,lag_3_day_4,lag_3_day_5,lag_3_day_6,lag_4_day_0,lag_4_day_1,lag_4_day_2,lag_4_day_3,lag_4_day_4,lag_4_day_5,lag_4_day_6
0,1,0,3137143.0,3248943.0,3135387.0,3142655.0,3227893.0,3312852.0,3212762.0,3269308.0,3308667.0,3311636.0,3369791.0,3196091.0,3258917.0,3289895.0,3515515.0,3508952.0,3535840.0,3132091.0,3351619.0,3263857.0,3115720.0,3227500.0,3221877.0,3223901.0,3116277.0,3010988.0,3071354.0,2974671.0,2918560.0,2909955.0,2925575.0,2971810.0,2948080.0,2913674.0,2953371.0
1,1,1,3269308.0,3308667.0,3311636.0,3369791.0,3196091.0,3258917.0,3289895.0,3515515.0,3508952.0,3535840.0,3132091.0,3351619.0,3263857.0,3115720.0,3227500.0,3221877.0,3223901.0,3116277.0,3010988.0,3071354.0,2974671.0,2918560.0,2909955.0,2925575.0,2971810.0,2948080.0,2913674.0,2953371.0,2943406.0,3035780.0,3048385.0,3026164.0,3191148.0,3125109.0,3150129.0
2,1,2,3515515.0,3508952.0,3535840.0,3132091.0,3351619.0,3263857.0,3115720.0,3227500.0,3221877.0,3223901.0,3116277.0,3010988.0,3071354.0,2974671.0,2918560.0,2909955.0,2925575.0,2971810.0,2948080.0,2913674.0,2953371.0,2943406.0,3035780.0,3048385.0,3026164.0,3191148.0,3125109.0,3150129.0,3149528.0,3249785.0,3374493.0,3401396.0,3461229.0,3484594.0,3397083.0
3,1,3,3227500.0,3221877.0,3223901.0,3116277.0,3010988.0,3071354.0,2974671.0,2918560.0,2909955.0,2925575.0,2971810.0,2948080.0,2913674.0,2953371.0,2943406.0,3035780.0,3048385.0,3026164.0,3191148.0,3125109.0,3150129.0,3149528.0,3249785.0,3374493.0,3401396.0,3461229.0,3484594.0,3397083.0,3444301.0,3409347.0,3552750.0,3381701.0,3389443.0,3345167.0,3523771.0
4,1,4,2918560.0,2909955.0,2925575.0,2971810.0,2948080.0,2913674.0,2953371.0,2943406.0,3035780.0,3048385.0,3026164.0,3191148.0,3125109.0,3150129.0,3149528.0,3249785.0,3374493.0,3401396.0,3461229.0,3484594.0,3397083.0,3444301.0,3409347.0,3552750.0,3381701.0,3389443.0,3345167.0,3523771.0,3695750.0,3762972.0,3823042.0,3849000.0,4016643.0,4166220.0,4117158.0
5,1,5,2943406.0,3035780.0,3048385.0,3026164.0,3191148.0,3125109.0,3150129.0,3149528.0,3249785.0,3374493.0,3401396.0,3461229.0,3484594.0,3397083.0,3444301.0,3409347.0,3552750.0,3381701.0,3389443.0,3345167.0,3523771.0,3695750.0,3762972.0,3823042.0,3849000.0,4016643.0,4166220.0,4117158.0,4328608.0,4389201.0,4466368.0,4457056.0,4493308.0,4452406.0,4358503.0
6,1,6,3149528.0,3249785.0,3374493.0,3401396.0,3461229.0,3484594.0,3397083.0,3444301.0,3409347.0,3552750.0,3381701.0,3389443.0,3345167.0,3523771.0,3695750.0,3762972.0,3823042.0,3849000.0,4016643.0,4166220.0,4117158.0,4328608.0,4389201.0,4466368.0,4457056.0,4493308.0,4452406.0,4358503.0,4528255.0,4598035.0,4632148.0,4660944.0,4689243.0,4720929.0,4749881.0
7,1,7,3444301.0,3409347.0,3552750.0,3381701.0,3389443.0,3345167.0,3523771.0,3695750.0,3762972.0,3823042.0,3849000.0,4016643.0,4166220.0,4117158.0,4328608.0,4389201.0,4466368.0,4457056.0,4493308.0,4452406.0,4358503.0,4528255.0,4598035.0,4632148.0,4660944.0,4689243.0,4720929.0,4749881.0,4661210.0,4674768.0,4699977.0,4857867.0,4839261.0,4892314.0,4955298.0
8,1,8,3695750.0,3762972.0,3823042.0,3849000.0,4016643.0,4166220.0,4117158.0,4328608.0,4389201.0,4466368.0,4457056.0,4493308.0,4452406.0,4358503.0,4528255.0,4598035.0,4632148.0,4660944.0,4689243.0,4720929.0,4749881.0,4661210.0,4674768.0,4699977.0,4857867.0,4839261.0,4892314.0,4955298.0,4994168.0,5031289.0,5029732.0,5132364.0,5119634.0,5264139.0,5208188.0
9,1,9,4328608.0,4389201.0,4466368.0,4457056.0,4493308.0,4452406.0,4358503.0,4528255.0,4598035.0,4632148.0,4660944.0,4689243.0,4720929.0,4749881.0,4661210.0,4674768.0,4699977.0,4857867.0,4839261.0,4892314.0,4955298.0,4994168.0,5031289.0,5029732.0,5132364.0,5119634.0,5264139.0,5208188.0,5224824.0,5245704.0,5141409.0,5215895.0,5263259.0,5175871.0,5275880.0


In [239]:
df_new["player_id"] = pp_df.loc[validation[0][1], "player_id"].values.copy()

In [240]:
df_new.head()

,week_-1_day_0_lags_8,week_0_day_0_lags_8,week_-1_day_0_lags_9,week_0_day_0_lags_9,week_-1_day_0_lags_10,week_0_day_0_lags_10,week_-1_day_0_lags_11,week_0_day_0_lags_11,week_-1_day_0_lags_12,week_0_day_0_lags_12,week_-1_day_0_lags_13,week_0_day_0_lags_13,week_-1_day_1_lags_8,week_0_day_1_lags_8,week_-1_day_1_lags_9,week_0_day_1_lags_9,week_-1_day_1_lags_10,week_0_day_1_lags_10,week_-1_day_1_lags_11,week_0_day_1_lags_11,week_-1_day_1_lags_12,week_0_day_1_lags_12,week_-1_day_1_lags_13,week_0_day_1_lags_13,week_-1_day_2_lags_8,week_0_day_2_lags_8,week_-1_day_2_lags_9,week_0_day_2_lags_9,week_-1_day_2_lags_10,week_0_day_2_lags_10,week_-1_day_2_lags_11,week_0_day_2_lags_11,week_-1_day_2_lags_12,week_0_day_2_lags_12,week_-1_day_2_lags_13,week_0_day_2_lags_13,week_-1_day_3_lags_8,week_0_day_3_lags_8,week_-1_day_3_lags_9,week_0_day_3_lags_9,week_-1_day_3_lags_10,week_0_day_3_lags_10,week_-1_day_3_lags_11,week_0_day_3_lags_11,week_-1_day_3_lags_12,week_0_day_3_lags_12,week_-1_day_3_lags_13,week_0_day_3_lags_13,week_-1_day_4_lags_8,week_0_day_4_lags_8,week_-1_day_4_lags_9,week_0_day_4_lags_9,week_-1_day_4_lags_10,week_0_day_4_lags_10,week_-1_day_4_lags_11,week_0_day_4_lags_11,week_-1_day_4_lags_12,week_0_day_4_lags_12,week_-1_day_4_lags_13,week_0_day_4_lags_13,week_-1_day_5_lags_8,week_0_day_5_lags_8,week_-1_day_5_lags_9,week_0_day_5_lags_9,week_-1_day_5_lags_10,week_0_day_5_lags_10,week_-1_day_5_lags_11,week_0_day_5_lags_11,week_-1_day_5_lags_12,week_0_day_5_lags_12,week_-1_day_5_lags_13,week_0_day_5_lags_13,week_-1_day_6_lags_8,week_0_day_6_lags_8,week_-1_day_6_lags_9,week_0_day_6_lags_9,week_-1_day_6_lags_10,week_0_day_6_lags_10,week_-1_day_6_lags_11,week_0_day_6_lags_11,week_-1_day_6_lags_12,week_0_day_6_lags_12,week_-1_day_6_lags_13,week_0_day_6_lags_13,player_id
0,-1.0,3137143.0,-1.0,3137143.0,-1.0,3137143.0,-1.0,3137143.0,-1.0,3137143.0,-1.0,3137143.0,-1.0,3248943.0,-1.0,3248943.0,-1.0,3248943.0,-1.0,3248943.0,-1.0,3248943.0,-1.0,3248943.0,-1.0,3135387.0,-1.0,3135387.0,-1.0,3135387.0,-1.0,3135387.0,-1.0,3135387.0,-1.0,3135387.0,-1.0,3142655.0,-1.0,3142655.0,-1.0,3142655.0,-1.0,3142655.0,-1.0,3142655.0,-1.0,3142655.0,-1.0,3227893.0,-1.0,3227893.0,-1.0,3227893.0,-1.0,3227893.0,-1.0,3227893.0,-1.0,3227893.0,-1.0,3312852.0,-1.0,3312852.0,-1.0,3312852.0,-1.0,3312852.0,-1.0,3312852.0,-1.0,3312852.0,-1.0,3212762.0,-1.0,3212762.0,-1.0,3212762.0,-1.0,3212762.0,-1.0,3212762.0,-1.0,3212762.0,1
1,-1.0,742514.0,-1.0,742514.0,-1.0,742514.0,-1.0,742514.0,-1.0,742514.0,-1.0,742514.0,-1.0,741715.0,-1.0,741715.0,-1.0,741715.0,-1.0,741715.0,-1.0,741715.0,-1.0,741715.0,-1.0,747924.0,-1.0,747924.0,-1.0,747924.0,-1.0,747924.0,-1.0,747924.0,-1.0,747924.0,-1.0,757681.0,-1.0,757681.0,-1.0,757681.0,-1.0,757681.0,-1.0,757681.0,-1.0,757681.0,-1.0,761528.0,-1.0,761528.0,-1.0,761528.0,-1.0,761528.0,-1.0,761528.0,-1.0,761528.0,-1.0,772174.0,-1.0,772174.0,-1.0,772174.0,-1.0,772174.0,-1.0,772174.0,-1.0,772174.0,-1.0,784590.0,-1.0,784590.0,-1.0,784590.0,-1.0,784590.0,-1.0,784590.0,-1.0,784590.0,2
2,-1.0,378507.0,-1.0,378507.0,-1.0,378507.0,-1.0,378507.0,-1.0,378507.0,-1.0,378507.0,-1.0,378681.0,-1.0,378681.0,-1.0,378681.0,-1.0,378681.0,-1.0,378681.0,-1.0,378681.0,-1.0,379722.0,-1.0,379722.0,-1.0,379722.0,-1.0,379722.0,-1.0,379722.0,-1.0,379722.0,-1.0,385732.0,-1.0,385732.0,-1.0,385732.0,-1.0,385732.0,-1.0,385732.0,-1.0,385732.0,-1.0,390694.0,-1.0,390694.0,-1.0,390694.0,-1.0,390694.0,-1.0,390694.0,-1.0,390694.0,-1.0,397944.0,-1.0,397944.0,-1.0,397944.0,-1.0,397944.0,-1.0,397944.0,-1.0,397944.0,-1.0,404118.0,-1.0,404118.0,-1.0,404118.0,-1.0,404118.0,-1.0,404118.0,-1.0,404118.0,3
3,-1.0,551679.0,-1.0,551679.0,-1.0,551679.0,-1.0,551679.0,-1.0,551679.0,-1.0,551679.0,-1.0,549743.0,-1.0,549743.0,-1.0,549743.0,-1.0,549743.0,-1.0,549743.0,-1.0,549743.0,-1.0,528290.0,-1.0,528290.0,-1.0,528290.0,-1.0,528290.0,-1.0,528290.0,-1.0,528290.0,-1.0,513586.0,-1.0,513586.0,-1.0,513586.0,-1.0,513586.0,-1.0,513586.0,-1.0,513586.0,-1.0,537107.0,-1.0,537107.0,-1.0,537107.0,-1.0,537107.0

In [196]:
pred_cols = ["week_-1_day_{}_lags_{}".format(day, lag) for day, lag in enumerate(best_days) ]
final_df = pd.melt(df_new, id_vars="player_id", value_vars=pred_cols)

In [197]:
pred_cols

['week_-1_day_0_lags_1',
 'week_-1_day_1_lags_1',
 'week_-1_day_2_lags_1',
 'week_-1_day_3_lags_6',
 'week_-1_day_4_lags_0',
 'week_-1_day_5_lags_1',
 'week_-1_day_6_lags_1']

In [198]:
d = ["2017-06-29", "2017-06-28", "2017-06-27", "2017-06-26", 
     "2017-06-25", "2017-06-24", "2017-06-23"]
dates = {}
for col, d in zip(pred_cols, d):
    dates[col] = d

In [199]:
final_df["variable"] = final_df["variable"].map(dates)
final_df.sort_values(["player_id", "variable"], inplace=True)
final_df.reset_index(drop=True, inplace=True)

In [200]:
final_df

,player_id,variable,value
0,1,2017-06-23,NaN
1,1,2017-06-24,NaN
2,1,2017-06-25,NaN
3,1,2017-06-26,NaN
4,1,2017-06-27,NaN
5,1,2017-06-28,NaN
6,1,2017-06-29,NaN
7,2,2017-06-23,NaN
8,2,2017-06-24,NaN
9,2,2017-06-25,NaN


In [ ]:
sample_subm["price"] = final_df["value"]

In [ ]:
sample_subm

In [ ]:
sample_subm[["id", "price"]].to_csv("linear_baseline_17500.csv", index=False)